# An Innovative System for Monitoring Emotional Health to Identify Individuals at Risk of Psychological Disturbances

In [1]:
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Data Reading

In [2]:
import json

with open('intents.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])
df

,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,morning,[Good morning],[Good morning. I hope you had a good night's s...
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night..."
...,...,...,...
75,fact-28,[What do I do if I'm worried about my mental h...,[The most important thing is to talk to someon...
76,fact-29,[How do I know if I'm unwell?],"[If your beliefs , thoughts , feelings or beha..."
77,fact-30,[How can I maintain social connections? What i...,"[A lot of people are alone right now, but we d..."
78,fact-31,[What's the difference between anxiety and str...,[Stress and anxiety are often used interchange...


Each tag contains several questions(patterns) and several answers(responses); Now I want to seperate these patterns based on their tags and responses and finally convert them to a DataFrame.

In [3]:
    dic = {"tag":[], "patterns":[], "responses":[]}
    for i in range(len(df)):
        ptrns = df[df.index == i]['patterns'].values[0]
        rspns = df[df.index == i]['responses'].values[0]
        tag = df[df.index == i]['tag'].values[0]
        for j in range(len(ptrns)):
            dic['tag'].append(tag)
            dic['patterns'].append(ptrns[j])
            dic['responses'].append(rspns)

    df = pd.DataFrame.from_dict(dic)
    df

,tag,patterns,responses
0,greeting,Hi,[Hello there. Tell me how are you feeling toda...
1,greeting,Hey,[Hello there. Tell me how are you feeling toda...
2,greeting,Is anyone there?,[Hello there. Tell me how are you feeling toda...
3,greeting,Hi there,[Hello there. Tell me how are you feeling toda...
4,greeting,Hello,[Hello there. Tell me how are you feeling toda...
...,...,...,...
227,fact-29,How do I know if I'm unwell?,"[If your beliefs , thoughts , feelings or beha..."
228,fact-30,How can I maintain social connections? What if...,"[A lot of people are alone right now, but we d..."
229,fact-31,What's the difference between anxiety and stress?,[Stress and anxiety are often used interchange...
230,fact-32,What's the difference between sadness and depr...,"[Sadness is a normal reaction to a loss, disap..."


In [4]:
df['tag'].unique()

array(['greeting', 'morning', 'afternoon', 'evening', 'night', 'goodbye',
       'thanks', 'no-response', 'neutral-response', 'about', 'skill',
       'creation', 'name', 'help', 'sad', 'stressed', 'worthless',
       'depressed', 'happy', 'casual', 'anxious', 'not-talking', 'sleep',
       'scared', 'death', 'understand', 'done', 'suicide', 'hate-you',
       'hate-me', 'default', 'jokes', 'repeat', 'wrong', 'stupid',
       'location', 'something-else', 'friends', 'ask', 'problem',
       'no-approach', 'learn-more', 'user-agree', 'meditation',
       'user-meditation', 'pandora-useful', 'user-advice',
       'learn-mental-health', 'mental-health-fact', 'fact-1', 'fact-2',
       'fact-3', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9',
       'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15',
       'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-20', 'fact-21',
       'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27',
       'fact-28', 'fact-29', '

# Data Preprocessing

Now I am going to apply some text preprocessing stuffs such as lowering, punctuation removing and then tokenize the patterns.

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['patterns'])
tokenizer.get_config()

{'num_words': None,
 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': None,
 'document_count': 232,
 'word_counts': '{"hi": 2, "hey": 2, "is": 13, "anyone": 2, "there": 5, "hello": 1, "howdy": 1, "hola": 1, "bonjour": 1, "konnichiwa": 1, "guten": 1, "tag": 1, "ola": 1, "good": 5, "morning": 1, "afternoon": 1, "evening": 1, "night": 1, "bye": 3, "see": 2, "you": 38, "later": 1, "goodbye": 1, "au": 1, "revoir": 1, "sayonara": 1, "ok": 4, "then": 1, "fare": 1, "thee": 1, "well": 2, "thanks": 2, "thank": 3, "that\'s": 3, "helpful": 1, "for": 10, "the": 10, "help": 6, "than": 1, "very": 3, "much": 4, "nothing": 3, "who": 6, "are": 15, "what": 29, "tell": 6, "me": 19, "more": 6, "about": 20, "yourself": 3, "your": 3, "name": 4, "should": 3, "i": 95, "call": 1, "what\'s": 3, "can": 16, "do": 14, "created": 2, "how": 8, "were": 2, "made": 1, "my": 13, "am": 17, "go": 4, "by": 1, "could": 2, "give": 1, "a": 12, "hand": 1, "ple

In [6]:
vacab_size = len(tokenizer.word_index)
print('number of unique words = ', vacab_size)

number of unique words =  303


In [7]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

ptrn2seq = tokenizer.texts_to_sequences(df['patterns'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['tag'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

X shape =  (232, 18)
y shape =  (232,)
num of classes =  80


In [8]:
X

array([[ 95,   0,   0, ...,   0,   0,   0],
       [ 96,   0,   0, ...,   0,   0,   0],
       [ 14,  97,  41, ...,   0,   0,   0],
       ...,
       [ 76,  19,  66, ...,   0,   0,   0],
       [ 76,  19,  66, ...,   0,   0,   0],
       [ 66,  67, 149, ...,   0,   0,   0]])

In [9]:
y

array([44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 55,  1, 10, 58, 43,
       43, 43, 43, 43, 43, 43, 43, 73, 73, 73, 73, 73, 60, 57,  0,  0,  0,
        0,  0,  0,  0,  0, 67,  5,  5,  5, 56, 56, 56, 48, 48, 48, 48, 48,
       48, 48, 65, 65, 65, 65, 65, 65, 65, 65, 70, 70, 70, 70, 70, 78, 78,
       78, 78, 78,  8,  8,  8,  8, 45, 45, 45, 45, 45, 45, 45,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  2,  2, 61, 61, 61, 61, 68, 68, 68,
       68, 68, 68, 66, 66, 66, 66,  6,  6,  6,  6,  6,  6, 74, 74, 74, 74,
       74, 74,  9,  9,  9,  9,  9, 72, 72, 72, 72, 72, 47, 47, 47, 46, 46,
       46,  7,  7,  7,  7,  7,  7,  7,  7, 49, 49, 64, 64, 64, 79, 79, 79,
       79, 71, 71, 71, 71, 52, 52, 52, 69, 69, 69, 69, 42,  3, 63, 63, 59,
       59, 59, 51, 51, 51, 76, 76, 53, 53, 77, 77, 62, 75, 75, 75, 50, 50,
       50, 54, 54, 11, 11, 22, 22, 33, 33, 37, 37, 37, 37, 38, 38, 39, 39,
       39, 40, 41, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26,
       27, 28, 29, 29, 30

# Build and Train Model

In [10]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.utils import plot_model

model = Sequential()
model.add(Input(shape=(X.shape[1])))
model.add(Embedding(input_dim=vacab_size+1, output_dim=100, mask_zero=True))
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32))
model.add(LayerNormalization())
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(np.unique(y)), activation="softmax"))
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

model.summary()
plot_model(model, show_shapes=True)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 100)           30400     
_________________________________________________________________
lstm (LSTM)                  (None, 18, 32)            17024     
_________________________________________________________________
layer_normalization (LayerNo (None, 18, 32)            64        
_________________________________________________________________
lstm_1 (LSTM)                (None, 18, 32)            8320      
________________________________________________________

In [11]:
model_history = model.fit(x=X,
                          y=y,
                          batch_size=10,
                          callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)],
                          epochs=50)

Epoch 1/50
232/232 [==============================] - 6s 27ms/sample - loss: 4.8090 - acc: 0.0302
Epoch 2/50
232/232 [==============================] - 2s 7ms/sample - loss: 3.5644 - acc: 0.1853
Epoch 3/50
232/232 [==============================] - 2s 7ms/sample - loss: 2.7842 - acc: 0.3233
Epoch 4/50
232/232 [==============================] - 2s 7ms/sample - loss: 2.1988 - acc: 0.4784
Epoch 5/50
232/232 [==============================] - 2s 7ms/sample - loss: 1.8934 - acc: 0.5560
Epoch 6/50
232/232 [==============================] - 2s 7ms/sample - loss: 1.5709 - acc: 0.6552
Epoch 7/50
232/232 [==============================] - 2s 7ms/sample - loss: 1.3557 - acc: 0.6897
Epoch 8/50
232/232 [==============================] - 2s 7ms/sample - loss: 1.0686 - acc: 0.7888
Epoch 9/50
232/232 [==============================] - 2s 7ms/sample - loss: 0.8648 - acc: 0.8405
Epoch 10/50
232/232 [==============================] - 2s 7ms/sample - loss: 0.6973 - acc: 0.8664
Epoch 11/50
232/232 [=======

232/232 [==============================] - 2s 7ms/sample - loss: 0.0953 - acc: 0.9914
Epoch 29/50
232/232 [==============================] - 2s 7ms/sample - loss: 0.0874 - acc: 0.9914
Epoch 30/50
232/232 [==============================] - 2s 8ms/sample - loss: 0.0718 - acc: 0.9957
Epoch 31/50
232/232 [==============================] - 2s 7ms/sample - loss: 0.0703 - acc: 0.9871
Epoch 32/50
232/232 [==============================] - 2s 7ms/sample - loss: 0.0644 - acc: 0.9914
Epoch 33/50
232/232 [==============================] - 2s 7ms/sample - loss: 0.0655 - acc: 0.9828
Epoch 34/50
232/232 [==============================] - 2s 7ms/sample - loss: 0.0633 - acc: 0.9914
Epoch 35/50
232/232 [==============================] - 2s 7ms/sample - loss: 0.0654 - acc: 0.9871
Epoch 36/50
232/232 [==============================] - 2s 7ms/sample - loss: 0.0576 - acc: 0.9957
Epoch 37/50
232/232 [==============================] - 2s 7ms/sample - loss: 0.0383 - acc: 1.0000
Epoch 38/50
232/232 [===========

# Model Testing

First we should apply some text preprocessing on the pattern that is passed to the function. Next we convert the text to vector of numbers and give it to model for prediction its tag; Finally based on the tag, we choose a answer(response) randomly and return it.

In [12]:
import re
import random

def generate_answer(pattern): 
    text = []
    txt = re.sub('[^a-zA-Z\']', ' ', pattern)
    txt = txt.lower()
    txt = txt.split()
    txt = " ".join(txt)
    text.append(txt)
        
    x_test = tokenizer.texts_to_sequences(text)
    x_test = np.array(x_test).squeeze()
    x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
    y_pred = model.predict(x_test)
    y_pred = y_pred.argmax()
    tag = lbl_enc.inverse_transform([y_pred])[0]
    responses = df[df['tag'] == tag]['responses'].values[0]

    print("you: {}".format(pattern))
    print("model: {}".format(random.choice(responses)))

In [13]:
generate_answer('Hi! How are you?')

you: Hi! How are you?
model: I'm sorry to hear that. I'm doing my best to help


In [14]:
generate_answer('Maybe I just didn\'t want to be born :)')

you: Maybe I just didn't want to be born :)
model: Talking about something really helps. If you're not ready to open up then that's ok. Just know that i'm here for you, whenever you need me.


In [15]:
generate_answer('help me:')

you: help me:
model: Sure. Tell me how can i assist you


In [16]:
generate_answer(':')

you: :
model: Please don't hesitate to talk to me.


In [17]:
def chatbot():
    print("Chatbot: Hi! I'm your friendly chatbot. How can I assist you today?")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['quit', 'exit', 'q', 'bye']:
            print("Chatbot: Goodbye!")
            break

        generate_answer(user_input)

if __name__ == "__main__":
    chatbot()

Chatbot: Hi! I'm your friendly chatbot. How can I assist you today?
You: hi hw r u


ValueError: `sequences` must be a list of iterables. Found non-iterable: 95